In [1]:
import geopandas as gpd
import folium
import pandas as pd
import json
import branca.colormap as cm
from shapely.geometry import shape

In [2]:
high = pd.read_csv('../쿼리/특수학급_고등학교_지역별.csv')
mid = pd.read_csv('../쿼리/특수학급_중학교_지역별.csv')
ele = pd.read_csv('../쿼리/특수학급_초등학교_지역별.csv')
sp = pd.read_csv('../쿼리/특수학급_특수학교.csv')
n_all = pd.read_csv('../쿼리/특수학급_특수학교제외_지역별.csv')
mi = pd.read_csv('../쿼리/특수학교_일반학교차.csv')

In [3]:
def prepare_data(dataframe):
    gdf = gpd.read_file('Seoul.geojson')

    gdf_dissolved = gdf.dissolve(by='sggnm').reset_index()

    dissolved_geojson = json.loads(gdf_dissolved.to_json())

    dataframe['구'] = dataframe['지역'].apply(lambda x: x.split()[-1])
    sp_agg = dataframe.groupby('구')['학생수(계)'].sum().reset_index()

    return dissolved_geojson, sp_agg

In [4]:
def prepare_data_2(dataframe):
    gdf = gpd.read_file('Seoul.geojson')

    gdf_dissolved = gdf.dissolve(by='sggnm').reset_index()

    dissolved_geojson = json.loads(gdf_dissolved.to_json())

    dataframe['구'] = dataframe['지역'].apply(lambda x: x.split()[-1])
    sp_agg = dataframe.groupby('구')['총특수학생수'].sum().reset_index()

    return dissolved_geojson, sp_agg

In [5]:
def prepare_data_3(dataframe):
    gdf = gpd.read_file('Seoul.geojson')

    gdf_dissolved = gdf.dissolve(by='sggnm').reset_index()

    dissolved_geojson = json.loads(gdf_dissolved.to_json())

    dataframe['구'] = dataframe['지역'].apply(lambda x: x.split()[-1])
    sp_agg = dataframe.groupby('구')['특수학생수차'].sum().reset_index()

    return dissolved_geojson, sp_agg

In [6]:
def visualize_student_population(dataframe, output_name):
    dissolved_geojson, sp_agg = prepare_data(dataframe)

    center_lat, center_lon = 37.5665, 126.9780
    m = folium.Map(location=[center_lat, center_lon], zoom_start=11, tiles='cartodbpositron')

    for feature in dissolved_geojson['features']:
        folium.GeoJson(
            feature,
            name=feature['properties']['sggnm'],
            style_function=lambda x: {
                'fillColor': 'none',
                'color': 'black',
                'weight': 1,
                'fillOpacity': 0,
            },
            tooltip=folium.GeoJsonTooltip(fields=['sggnm'], labels=False, sticky=False)
        ).add_to(m)

    min_students = sp_agg['학생수(계)'].min()
    max_students = sp_agg['학생수(계)'].max()
    color_scale = cm.LinearColormap(['floralwhite', 'wheat', 'orange', 'darkgoldenrod'], vmin=min_students, vmax=max_students)

    for _, row in sp_agg.iterrows():
        district = row['구']
        student_count = row['학생수(계)']
        
        for feature in dissolved_geojson['features']:
            if feature['properties']['sggnm'] == district:
                folium.GeoJson(
                    feature,
                    name=district,
                    style_function=lambda x, students=student_count: {
                        'fillColor': color_scale(students),
                        'color': 'black',
                        'weight': 1,
                        'fillOpacity': 0.7,
                    },
                    tooltip=folium.GeoJsonTooltip(fields=['sggnm'], labels=False, sticky=False)
                ).add_to(m)

    for feature in dissolved_geojson['features']:
        district_name = feature['properties']['sggnm']
        geometry = shape(feature['geometry'])
        centroid = geometry.centroid
        folium.Marker(
            location=[centroid.y, centroid.x],
            icon=folium.DivIcon(html=f'''
                <div style="
                    display: inline-block;
                    max-width: 200px;
                    font-size: 10px; 
                    color: black; 
                    background-color: #f0f0f0; 
                    padding: 2px; 
                    border-radius: 3px; 
                    text-align: center; 
                    white-space: nowrap;">
                    {district_name}
                </div>
            ''')
        ).add_to(m)

    color_scale.caption = '특수교육 대상자 [단위 : 명]'
    m.add_child(color_scale)

    output_file = f'vis_{output_name}.html'
    m.save(output_file)

    return output_file

In [7]:
def visualize_student_population_2(dataframe, output_name):
    dissolved_geojson, sp_agg = prepare_data_2(dataframe)

    center_lat, center_lon = 37.5665, 126.9780
    m = folium.Map(location=[center_lat, center_lon], zoom_start=11, tiles='cartodbpositron')

    for feature in dissolved_geojson['features']:
        folium.GeoJson(
            feature,
            name=feature['properties']['sggnm'],
            style_function=lambda x: {
                'fillColor': 'none',
                'color': 'black',
                'weight': 1,
                'fillOpacity': 0,
            },
            tooltip=folium.GeoJsonTooltip(fields=['sggnm'], labels=False, sticky=False)
        ).add_to(m)

    min_students = sp_agg['총특수학생수'].min()
    max_students = sp_agg['총특수학생수'].max()
    color_scale = cm.LinearColormap(['floralwhite', 'wheat', 'orange', 'darkgoldenrod'], vmin=min_students, vmax=max_students)

    for _, row in sp_agg.iterrows():
        district = row['구']
        student_count = row['총특수학생수']
        
        for feature in dissolved_geojson['features']:
            if feature['properties']['sggnm'] == district:
                folium.GeoJson(
                    feature,
                    name=district,
                    style_function=lambda x, students=student_count: {
                        'fillColor': color_scale(students),
                        'color': 'black',
                        'weight': 1,
                        'fillOpacity': 0.7,
                    },
                    tooltip=folium.GeoJsonTooltip(fields=['sggnm'], labels=False, sticky=False)
                ).add_to(m)

    for feature in dissolved_geojson['features']:
        district_name = feature['properties']['sggnm']
        geometry = shape(feature['geometry'])
        centroid = geometry.centroid
        folium.Marker(
            location=[centroid.y, centroid.x],
            icon=folium.DivIcon(html=f'''
                <div style="
                    display: inline-block;
                    max-width: 200px;
                    font-size: 10px; 
                    color: black; 
                    background-color: #f0f0f0; 
                    padding: 2px; 
                    border-radius: 3px; 
                    text-align: center; 
                    white-space: nowrap;">
                    {district_name}
                </div>
            ''')
        ).add_to(m)

    color_scale.caption = '특수 학생 비율'
    m.add_child(color_scale)

    output_file = f'vis_{output_name}.html'
    m.save(output_file)

    return output_file

In [8]:
def visualize_student_population_3(dataframe, output_name):
    dissolved_geojson, sp_agg = prepare_data_3(dataframe)

    center_lat, center_lon = 37.5665, 126.9780
    m = folium.Map(location=[center_lat, center_lon], zoom_start=11, tiles='cartodbpositron')

    for feature in dissolved_geojson['features']:
        folium.GeoJson(
            feature,
            name=feature['properties']['sggnm'],
            style_function=lambda x: {
                'fillColor': 'none',
                'color': 'black',
                'weight': 1,
                'fillOpacity': 0,
            },
            tooltip=folium.GeoJsonTooltip(fields=['sggnm'], labels=False, sticky=False)
        ).add_to(m)

    min_students = sp_agg['특수학생수차'].min()
    max_students = sp_agg['특수학생수차'].max()
    color_scale = cm.LinearColormap(['floralwhite', 'wheat', 'orange', 'darkgoldenrod'], vmin=min_students, vmax=max_students)

    for _, row in sp_agg.iterrows():
        district = row['구']
        student_count = row['특수학생수차']
        
        for feature in dissolved_geojson['features']:
            if feature['properties']['sggnm'] == district:
                folium.GeoJson(
                    feature,
                    name=district,
                    style_function=lambda x, students=student_count: {
                        'fillColor': color_scale(students),
                        'color': 'black',
                        'weight': 1,
                        'fillOpacity': 0.7,
                    },
                    tooltip=folium.GeoJsonTooltip(fields=['sggnm'], labels=False, sticky=False)
                ).add_to(m)

    for feature in dissolved_geojson['features']:
        district_name = feature['properties']['sggnm']
        geometry = shape(feature['geometry'])
        centroid = geometry.centroid
        folium.Marker(
            location=[centroid.y, centroid.x],
            icon=folium.DivIcon(html=f'''
                <div style="
                    display: inline-block;
                    max-width: 200px;
                    font-size: 10px; 
                    color: black; 
                    background-color: #f0f0f0; 
                    padding: 2px; 
                    border-radius: 3px; 
                    text-align: center; 
                    white-space: nowrap;">
                    {district_name}
                </div>
            ''')
        ).add_to(m)

    color_scale.caption = '특수 학생 비율'
    m.add_child(color_scale)

    output_file = f'vis_{output_name}.html'
    m.save(output_file)

    return output_file

In [9]:
visualize_student_population(sp, 'Special')

'vis_Special.html'

In [10]:
visualize_student_population_2(ele, 'Elementary')

'vis_Elementary.html'

In [11]:
visualize_student_population_2(mid, 'Middle')

'vis_Middle.html'

In [12]:
visualize_student_population_2(high, 'High')

'vis_High.html'

In [13]:
visualize_student_population_2(n_all, 'NotSpecial')

'vis_NotSpecial.html'

In [14]:
visualize_student_population_3(mi, 'mi')

'vis_mi.html'